In [1]:
# Imports
import pandas as pd
import numpy as np
import math
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
games_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/games.csv')
players_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/players.csv')
plays_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/full_play_2.csv')
week1r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week1.csv')
week2r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week2.csv')
week3r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week3.csv')
week4r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week4.csv')
week5r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week5.csv')
week6r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week6.csv')
week7r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week7.csv')
week8r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week8.csv')
week9r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week9.csv')
week10r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week10.csv')
week11r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week11.csv')
week12r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week12.csv')
week13r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week13.csv')
week14r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week14.csv')
week15r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week15.csv')
week16r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week16.csv')
week17r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week17.csv')

In [3]:
# Reduce the weeks data to pass_arrived data
def week_reduce(df):
    df = df[(df['event'] == 'pass_arrived')]
    return df

week1_df = week_reduce(week1r_df)
week2_df = week_reduce(week2r_df)
week3_df = week_reduce(week3r_df)
week4_df = week_reduce(week4r_df)
week5_df = week_reduce(week5r_df)
week6_df = week_reduce(week6r_df)
week7_df = week_reduce(week7r_df)
week8_df = week_reduce(week8r_df)
week9_df = week_reduce(week9r_df)
week10_df = week_reduce(week10r_df)
week11_df = week_reduce(week11r_df)
week12_df = week_reduce(week12r_df)
week13_df = week_reduce(week13r_df)
week14_df = week_reduce(week14r_df)
week15_df = week_reduce(week15r_df)
week16_df = week_reduce(week16r_df)
week17_df = week_reduce(week17r_df)

In [4]:
# Create a new column identifying offensive and defensive players
def weekly_conv(df):
    df['defPlayer'] = np.where((df['position'] == 'SS') | 
                                 (df['position'] == 'FS') |
                                 (df['position'] == 'MLB') | 
                                 (df['position'] == 'CB') |
                                 (df['position'] == 'LB') | 
                                 (df['position'] == 'OLB') |
                                 (df['position'] == 'ILB') | 
                                 (df['position'] == 'DL') |
                                 (df['position'] == 'DB') | 
                                 (df['position'] == 'NT') |
                                 (df['position'] == 'S') | 
                                 (df['position'] == 'DE') |
                                 (df['position'] == 'DT'),
                                 'D', 'O')
    
    return df.sort_values(by=['gameId', 'playId', 'frameId', 'team'])

w1_df = weekly_conv(week1_df)
w2_df = weekly_conv(week2_df)
w3_df = weekly_conv(week3_df)
w4_df = weekly_conv(week4_df)
w5_df = weekly_conv(week5_df)
w6_df = weekly_conv(week6_df)
w7_df = weekly_conv(week7_df)
w8_df = weekly_conv(week8_df)
w9_df = weekly_conv(week9_df)
w10_df = weekly_conv(week10_df)
w11_df = weekly_conv(week11_df)
w12_df = weekly_conv(week12_df)
w13_df = weekly_conv(week13_df)
w14_df = weekly_conv(week14_df)
w15_df = weekly_conv(week15_df)
w16_df = weekly_conv(week16_df)
w17_df = weekly_conv(week17_df)

In [5]:
# Set the column names for the result df
column_names = ['weekId', 'gameId', 'playId', 'oPlayer', 'dPlayer', 'route', 'endDist', 'endTime']
result_df = pd.DataFrame(columns = column_names)

In [6]:
# Create a result df containing the closest defender to each receiver running a route
def get_closest_dplayer(wid, gid, pid, eid, r_df):
    
    # print(wid, gid, pid, eid)
    
    if wid == 1:
        w_df = w1_df
    elif wid == 2:
        w_df = w2_df
    elif wid == 3:
        w_df = w3_df
    elif wid == 4:
        w_df = w4_df
    elif wid == 5:
        w_df = w5_df
    elif wid == 6:
        w_df = w6_df
    elif wid == 7:
        w_df = w7_df
    elif wid == 8:
        w_df = w8_df
    elif wid == 9:
        w_df = w9_df
    elif wid == 10:
        w_df = w10_df
    elif wid == 11:
        w_df = w11_df
    elif wid == 12:
        w_df = w12_df
    elif wid == 13:
        w_df = w13_df
    elif wid == 14:
        w_df = w14_df
    elif wid == 15:
        w_df = w15_df
    elif wid == 16:
        w_df = w16_df
    elif wid == 17:
        w_df = w17_df
    
    # Create a frame df
    f_df = w_df.loc[(w_df['playId'] == pid) & (w_df['gameId'] == gid) & (w_df['event'] == eid)]
    
    # Create a df for defenders (d_df) and receivers (o_df)
    d_df = f_df.loc[(f_df['defPlayer'] == 'D') & 
                    (f_df['displayName'] != 'Football')]
    o_df = f_df.loc[(f_df['defPlayer'] == 'O') & 
                    (f_df['displayName'] != 'Football') & 
                    (f_df['position'] != 'QB') & 
                    (f_df['route'].notnull())]
    
    # Test for empty df's
    if d_df.empty | o_df.empty:
        return r_df

    # For each offensive reciever find closest defender
    for i in o_df.index:
        
        # Offensive player location, name and route
        opos_x = o_df.at[i, 'x']
        opos_y = o_df.at[i, 'y']
        o_player = o_df.at[i,'displayName']
        route = o_df.at[i, 'route']
        
        # Calculate the distance
        for i in d_df.index:
            d_df.at[i, 'bd'] = math.sqrt(((opos_x - d_df.at[i, 'x'])**2) + 
                                         ((opos_y - d_df.at[i, 'y'])**2))
        
        d_df['rank'] = d_df['bd'].rank()
        if d_df.loc[d_df['rank'] == 1]['displayName'].empty:
            dist = 100
        else:
            d_player = d_df.loc[d_df['rank'] == 1]['displayName'].item()
            
            p_time = d_df.loc[d_df['rank'] == 1]['time'].item()
            dt_string = p_time[11:23]
            dt_obj = datetime.datetime.strptime(dt_string, '%H:%M:%S.%f')
            
            dist = d_df.loc[d_df['rank'] == 1]['bd'].item()
            row = {'weekId': wid, 'gameId': gid, 'playId': pid, 'oPlayer': o_player, 
                   'dPlayer': d_player, 'route': route, 'endDist': dist, 'endTime': dt_obj}
            r_df = r_df.append(row, ignore_index = True)
            d_df.drop(d_df[d_df['displayName'] == d_player].index, inplace = True)
            
    return r_df

In [7]:
for i in plays_df.index:
    
    result_df = get_closest_dplayer(plays_df.at[i, 'week'], 
                                    plays_df.at[i, 'gameId'],
                                    plays_df.at[i, 'playId'],
                                    'pass_arrived',
                                    result_df)

result_df.head(20) 

,weekId,gameId,playId,oPlayer,dPlayer,route,endDist,endTime
0,1,2018090600,75,Julio Jones,Malcolm Jenkins,HITCH,2.957871,1900-01-01 01:07:19.200
1,1,2018090600,75,Mohamed Sanu,Nate Gerry,HITCH,5.000090,1900-01-01 01:07:19.200
2,1,2018090600,75,Austin Hooper,Ronald Darby,OUT,1.360588,1900-01-01 01:07:19.200
3,1,2018090600,75,Ricky Ortiz,Corey Graham,FLAT,7.915112,1900-01-01 01:07:19.200
4,1,2018090600,146,Devonta Freeman,Sidney Jones,FLAT,10.383549,1900-01-01 01:09:13.200
5,1,2018090600,146,Justin Hardy,Kamu Grugier-Hill,CROSS,3.198203,1900-01-01 01:09:13.200
6,1,2018090600,146,Austin Hooper,Malcolm Jenkins,FLAT,1.646147,1900-01-01 01:09:13.200
7,1,2018090600,146,Marvin Hall,Ronald Darby,HITCH,3.465776,1900-01-01 01:09:13.200
8,1,2018090600,146,Calvin Ridley,Jordan Hicks,HITCH,1.891719,1900-01-01 01:09:13.200
9,1,2018090600,190,Julio Jones,Ronald Darby,CORNER,3.898949,1900-01-01 01:10:40.099


In [9]:
result_df.to_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/final/distance.csv')